In [2]:
import sys
sys.path.append("/media/arclabdl1/HD1/Linjun/mpc-mpnet-py/mpnet/exported")

from exported.export_mpnet_external_small_model import KMPNet, load_func, Encoder, MLP
mpnet = KMPNet(total_input_size=8, AE_input_size=32, mlp_input_size=40, output_size=4, CAE=Encoder, MLP=MLP, loss_f=None)
load_func(mpnet, '/media/arclabdl1/HD1/YLmiao/results/KMPnet_res/cartpole_obs_4_lr0.010000_Adagrad_step_200/kmpnet_epoch_3150_direction_0_step_200.pkl')

length of the output of one encoder
343


In [25]:
from dataset.dataset import get_loader_cost
system_env = "sst_envs"
system = "cartpole_obs"
batch_size = 1024
setup = "default_norm"
label_type = "cost_to_go"
data_type = 'path_data'
ep = 2500
train_loader, test_loader = get_loader_cost(system_env, 
                               system, 
                               batch_size=batch_size, 
                               setup=setup, 
                               label_type=label_type, 
                               data_type=data_type)

In [31]:
import torch
import numpy as np
from networks.costnet import CostNet 
from tqdm.notebook import tqdm
costnet = CostNet(ae_input_size=32, ae_output_size=32, in_channels=1, state_size=4, encoder=mpnet.encoder).cuda()
costnet.load_state_dict(torch.load('output/{}/{}/cost_to_go/ep{}.pth'.format(system, setup, ep)))


env_vox = torch.from_numpy(np.load('{}/{}_env_vox.npy'.format(system_env, system))).float()

if torch.cuda.is_available():
    costnet = costnet.cuda()
    env_vox = env_vox.cuda()

eval_output_list = []

for data, label in tqdm(test_loader):
    with torch.no_grad():
        if torch.cuda.is_available():
            data = data.cuda()
            label = label.cuda()                
        inputs = data[:,1:]
        envs = env_vox[(data[:,0]).long()]
        output = costnet(inputs, envs) 
        eval_output_list.append(output-label)


In [32]:
eval_output_list

[tensor([[ 0.0597],
         [ 0.3551],
         [ 0.2034],
         ...,
         [-1.1227],
         [-1.0556],
         [ 0.0295]], device='cuda:0'),
 tensor([[ 0.5430],
         [ 0.1351],
         [ 0.0649],
         ...,
         [-0.0049],
         [ 1.0343],
         [-0.0448]], device='cuda:0'),
 tensor([[-0.1043],
         [ 0.3451],
         [-0.3137],
         ...,
         [ 0.1964],
         [ 0.2750],
         [ 0.1847]], device='cuda:0'),
 tensor([[ 0.2022],
         [-0.0318],
         [-0.1553],
         ...,
         [-0.1857],
         [-0.1837],
         [-0.0247]], device='cuda:0'),
 tensor([[-0.3845],
         [-0.4453],
         [ 0.2728],
         ...,
         [ 0.2481],
         [-0.0768],
         [-0.1233]], device='cuda:0'),
 tensor([[ 0.3210],
         [ 0.0719],
         [-0.5187],
         ...,
         [ 0.2688],
         [ 0.2457],
         [-0.1441]], device='cuda:0'),
 tensor([[ 0.0876],
         [-0.6683],
         [ 0.3547],
         ...,
        